# ImageNet Analysis
> Analysis of data for the ImageNet benchmark

- toc: true 
- badges: true
- comments: true
- categories: [performance, ImageNet]
- image: images/imagenet_logo.png

# Theoretical Analysis of ImageNet

### Rooflines

Combining application requirements with hardware platform characteristics can be leveraged for performance predictions using UCB’s roofline models. Using assumptions for where weights, activation tensors, and state of a neural network are stored, combined with the size of the datatypes used, allow us to derive the arithmetic intensity of a neural network during inference. Combined with the roofline for a given hardware platform, we can provide insight as to whether a neural network will be memory or compute bound and guidance for what is theoretically possible in regards to its throughput.

In [1]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load dataset and plot it
rooflines(pd.read_csv("data/processed_csv/rooflines_hardware_neural_networks.csv"), 'imagenet')

alt.LayerChart(...)

### Performance Prediction

In [2]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load imagenet dataset and plot it
heatmap(pd.read_csv("data/processed_csv/performance_prediction_imagenet.csv"), 'lightgrey','Performance Prediction for Imagenet')

alt.LayerChart(...)

# Experimental Data Analysis

### Overview of All Measurements for MNIST

In [3]:
#hide_input
%run scripts/altair_plots.py  #get table with the experiments overview
tableOverviewExperiments(['data/overview_experiments_imagenet.csv'])

In [4]:
#hide
import pandas as pd
import numpy as np
import altair as alt

W = 600
H = 480

csv_path = "./data/cleaned_csv/backup.csv"

In [5]:
#hide
#%writefile scripts/utils.py
def norm_by_group(df, column, group_col):
    """ Normalizes pandas series by group """
    df["norm-"+column] = df.groupby(group_col)[column].apply(lambda x: (x / x.max()))
    return df

def select_color(sel, column):
    """ Easy way to set colors based on selection for altair plots
    """
    return alt.condition(sel, 
                      alt.Color(column),
                      alt.value('lightgray'))

def get_pareto_df(df, groupcol, xcol, ycol):
    pareto_line_df = df.groupby(groupcol)[xcol].max().to_frame("x")
    pareto_line_df['y'] = df.groupby(groupcol)[ycol].agg(lambda x: x.value_counts().index[0])
    pareto_line_df.sort_values('y', ascending=False, inplace=True)
    pareto_line_df['x'] = pareto_line_df.x.cummax()
    pareto_line_df.drop_duplicates('x', keep='first', inplace=True)
    pareto_line_df['group'] = pareto_line_df.index
    return pareto_line_df

def label_point(x, y, val, ax, rot=0):
    """ from https://stackoverflow.com/questions/46027653/adding-labels-in-x-y-scatter-plot-with-seaborn"""
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x']+.02, point['y'], str(point['val']), rotation=rot)

In [6]:
#hide
master_df = pd.read_csv(csv_path)
is_maxp = lambda row: row.HWType != "TX2" or row["Op mode"].split(",")[0] == "maxp"
maxp_df = master_df[master_df.apply(is_maxp, axis=1)]
imagenet_df = maxp_df[maxp_df.NN_Topology.isin(['GNv1','RN50','MNv1']) & maxp_df["top1 [%]"].notna() & maxp_df['lat-comp'].notna()]
bad_precisions = ["FP"+str(i) for i in range(17,24)]
imagenet_df.Precision = imagenet_df.Precision.apply(lambda x: 'FP16' if x in bad_precisions else x)
imagenet_df["hw_quant_prun_net"] = imagenet_df.apply(lambda r: "_".join([r.HWType, r.Precision, r.PruningFactor, r.NN_Topology]), axis=1)
imagenet_df["PruningFactor"] = imagenet_df["PruningFactor"].str.strip("%").astype(float)
norm_by_group(imagenet_df, "lat-comp", "NN_Topology");
imagenet_df["quant_model"] = imagenet_df.Precision + '_' + imagenet_df.HWType
imagenet_df.rename(columns={"top1 [%]": "top1"}, inplace=True)
imagenet_df["hw_precision_net_prun"] = imagenet_df.apply(lambda r: "_".join([r.HWType, r.Precision, r.NN_Topology, str(r.PruningFactor)]), axis=1)

/home/zach/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/zach/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/zach/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

### Line Plot

In [7]:
#hide_input

sel = alt.selection_multi(fields=["hw_quant_prun_net"], bind="legend")
fig25_dot = alt.Chart(imagenet_df).mark_point().encode(
    x='lat-comp',
    y=alt.Y('fps-comp', scale=alt.Scale(type="log")),
    color=select_color(sel, 'hw_quant_prun_net:N'),
    tooltip=['Op mode', 'fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
)
fig25_line = alt.Chart(imagenet_df).mark_line().encode(
    x='lat-comp',
    y='fps-comp',
    color=select_color(sel, 'hw_quant_prun_net:N'),
    tooltip=['Op mode', 'fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
)

fig = (fig25_dot+fig25_line).properties(
    title="Latency versus Performance for Pruned and Quantized Imagenet Classifier Variants",
    width=W,
    height=H,
).add_selection(sel).interactive()

fig

alt.LayerChart(...)

### Boxplots

In [8]:
#hide_input
box1 = alt.Chart(imagenet_df).mark_boxplot().encode(
    x='PruningFactor:O',
    y=alt.Y("lat-comp", scale=alt.Scale(type="log")),
    color='PruningFactor:O',
).facet(column="quant_model").properties(
    title="Latency by Hardware/Framework and Pruning for imagenet"
).interactive()
box1

alt.FacetChart(...)

In [9]:
#hide_input
box1 = alt.Chart(imagenet_df).mark_boxplot().encode(
    x='PruningFactor:O',
    y=alt.Y("fps-comp", scale=alt.Scale(type="log")),
    color='PruningFactor:O',
).facet(column="quant_model").properties(
    title="Throughput by Hardware/Framework and Pruning for imagenet",
).interactive()
box1

alt.FacetChart(...)

## Pareto Graphs

In [10]:
#hide_input
imagenet_pareto = get_pareto_df(imagenet_df, 'hw_precision_net_prun', 'fps-comp', 'top1')
imagenet_lines = alt.Chart(imagenet_df).mark_line(point=True).encode(
    x="fps-comp",
    y=alt.Y("top1:Q", scale=alt.Scale(zero=False)),
    color=alt.Color("hw_precision_net_prun", legend=alt.Legend(columns=1)),
    tooltip=["HWType", "Precision", "PruningFactor", "batch/thread/stream", "top1", "fps-comp"],
)
imagenet_pareto_plot = alt.Chart(imagenet_pareto).mark_line().encode(
    x="x",
    y=alt.Y("y", scale=alt.Scale(zero=False)),
)
(imagenet_lines+imagenet_pareto_plot).interactive().properties(
    width=W,
    height=H,
    title="ImageNet Cassification Design Space: Accuracy versus Performance"
)

alt.LayerChart(...)

In [11]:
# Measurements

In [12]:
#hide
imagenet_df.to_csv('data/processed_csv/experimental_data_imagenet.csv', index = False)
imagenet_df.head()

,NN_Topology,HWType,Precision,Op mode,batch/thread/stream,lat-comp,fps-system,fps-comp,tp-system,tp-comp,...,BasePWR [W],IdlePWR [W],FullPwr [W],GOPS,PruningFactor,level,hw_quant_prun_net,norm-lat-comp,quant_model,hw_precision_net_prun
269,MNv1,TPU,INT8,slow,1,4.08249,127.2560,244.949,398.31128,766.69037,...,0.253,0.253,0.462,NaN,100.0,l3,TPU_INT8_100%_MNv1,1.000000,INT8_TPU,TPU_INT8_MNv1_100.0
270,MNv1,TPU,INT8,fast,1,2.57047,166.5330,389.034,521.24829,1217.67642,...,0.253,0.253,0.532,NaN,100.0,l3,TPU_INT8_100%_MNv1,0.629633,INT8_TPU,TPU_INT8_MNv1_100.0
399,GNv1,TPU,INT8,slow,1,5.72131,99.7410,174.785,312.18933,547.07705,...,0.253,0.253,0.463,NaN,100.0,l3,TPU_INT8_100%_GNv1,0.006099,INT8_TPU,TPU_INT8_GNv1_100.0
400,GNv1,TPU,INT8,fast,1,3.64852,135.0870,274.084,422.82231,857.88292,...,0.253,0.253,0.538,NaN,100.0,l3,TPU_INT8_100%_GNv1,0.003890,INT8_TPU,TPU_INT8_GNv1_100.0
433,GNv1,TX2,FP16,maxp,1,6.16337,99.8245,169.338,312.45100,530.02900,...,1.800,4.700,8.070,NaN,100.0,l3,TX2_FP16_100%_GNv1,0.006571,FP16_TX2,TX2_FP16_GNv1_100.0
